In [17]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [20]:


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=2048,pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=2048,pin_memory=True)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

Hyperparameters
===============

Hyperparameters are adjustable parameters that let you control the model
optimization process. Different hyperparameter values can impact model
training and convergence rates ([read
more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)
about hyperparameter tuning)

We define the following hyperparameters for training:

:   -   **Number of Epochs** - the number times to iterate over the
        dataset
    -   **Batch Size** - the number of data samples propagated through
        the network before the parameters are updated
    -   **Learning Rate** - how much to update models parameters at each
        batch/epoch. Smaller values yield slow learning speed, while
        large values may result in unpredictable behavior during
        training.


In [38]:
learning_rate1 = 1e-3
learning_rate2 = 3*3e-3
learning_rate3 = 1e-3*1e-2*1

batch_size = 64
epochs = 10

Optimization Loop
=================

Once we set our hyperparameters, we can then train and optimize our
model with an optimization loop. Each iteration of the optimization loop
is called an **epoch**.

Each epoch consists of two main parts:

:   -   **The Train Loop** - iterate over the training dataset and try
        to converge to optimal parameters.
    -   **The Validation/Test Loop** - iterate over the test dataset to
        check if model performance is improving.

Let\'s briefly familiarize ourselves with some of the concepts used in
the training loop. Jump ahead to see the
`full-impl-label`{.interpreted-text role="ref"} of the optimization
loop.

Loss Function
-------------

When presented with some training data, our untrained network is likely
not to give the correct answer. **Loss function** measures the degree of
dissimilarity of obtained result to the target value, and it is the loss
function that we want to minimize during training. To calculate the loss
we make a prediction using the inputs of our given data sample and
compare it against the true data label value.

Common loss functions include
[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)
(Mean Square Error) for regression tasks, and
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
(Negative Log Likelihood) for classification.
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model\'s output logits to `nn.CrossEntropyLoss`, which will
normalize the logits and compute the prediction error.


In [23]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()


Optimizer
=========

Optimization is the process of adjusting model parameters to reduce
model error in each training step. **Optimization algorithms** define
how this process is performed (in this example we use Stochastic
Gradient Descent). All optimization logic is encapsulated in the
`optimizer` object. Here, we use the SGD optimizer; additionally, there
are many [different
optimizers](https://pytorch.org/docs/stable/optim.html) available in
PyTorch such as ADAM and RMSProp, that work better for different kinds
of models and data.

We initialize the optimizer by registering the model\'s parameters that
need to be trained, and passing in the learning rate hyperparameter.


In [24]:
optimizer1 = torch.optim.SGD(model.parameters(), lr=learning_rate1)
optimizer2 = torch.optim.Adam(model.parameters(), lr=learning_rate1)
optimizer3 = torch.optim.AdamW(model.parameters(), lr=learning_rate1)

Inside the training loop, optimization happens in three steps:

:   -   Call `optimizer.zero_grad()` to reset the gradients of model
        parameters. Gradients by default add up; to prevent
        double-counting, we explicitly zero them at each iteration.
    -   Backpropagate the prediction loss with a call to
        `loss.backward()`. PyTorch deposits the gradients of the loss
        w.r.t. each parameter.
    -   Once we have our gradients, we call `optimizer.step()` to adjust
        the parameters by the gradients collected in the backward pass.


Full Implementation
===================

We define `train_loop` that loops over our optimization code, and
`test_loop` that evaluates the model\'s performance against our test
data.


In [33]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X= X.to(device)
        y =y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    best_loss = float("inf")
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X= X.to(device)
            y =y.to(device)
            pred = model(X)
            running_loss= loss_fn(pred, y).item()
            test_loss += running_loss
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            if (running_loss) <= best_loss:
              best_loss = running_loss
              torch.save(model.state_dict(),"model_weight.pth")
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to
`train_loop` and `test_loop`. Feel free to increase the number of epochs
to track the model\'s improving performance.


In [26]:

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer1)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294195  [ 2048/60000]
Test Error: 
 Accuracy: 15.2%, Avg loss: 2.291086 

Epoch 2
-------------------------------
loss: 2.289691  [ 2048/60000]
Test Error: 
 Accuracy: 16.9%, Avg loss: 2.286611 

Epoch 3
-------------------------------
loss: 2.285214  [ 2048/60000]
Test Error: 
 Accuracy: 18.1%, Avg loss: 2.282155 

Epoch 4
-------------------------------
loss: 2.280751  [ 2048/60000]
Test Error: 
 Accuracy: 18.9%, Avg loss: 2.277712 

Epoch 5
-------------------------------
loss: 2.276299  [ 2048/60000]
Test Error: 
 Accuracy: 19.8%, Avg loss: 2.273275 

Epoch 6
-------------------------------
loss: 2.271853  [ 2048/60000]
Test Error: 
 Accuracy: 21.0%, Avg loss: 2.268838 

Epoch 7
-------------------------------
loss: 2.267401  [ 2048/60000]
Test Error: 
 Accuracy: 22.4%, Avg loss: 2.264393 

Epoch 8
-------------------------------
loss: 2.262943  [ 2048/60000]
Test Error: 
 Accuracy: 23.7%, Avg loss: 2.259937 

Epoch 9
----------------

In [27]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer2)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.200982  [ 2048/60000]
Test Error: 
 Accuracy: 77.7%, Avg loss: 0.609101 

Epoch 2
-------------------------------
loss: 0.578909  [ 2048/60000]
Test Error: 
 Accuracy: 81.5%, Avg loss: 0.504576 

Epoch 3
-------------------------------
loss: 0.459445  [ 2048/60000]
Test Error: 
 Accuracy: 83.5%, Avg loss: 0.461036 

Epoch 4
-------------------------------
loss: 0.411737  [ 2048/60000]
Test Error: 
 Accuracy: 84.5%, Avg loss: 0.433644 

Epoch 5
-------------------------------
loss: 0.382717  [ 2048/60000]
Test Error: 
 Accuracy: 85.4%, Avg loss: 0.405628 

Epoch 6
-------------------------------
loss: 0.353468  [ 2048/60000]
Test Error: 
 Accuracy: 86.1%, Avg loss: 0.388474 

Epoch 7
-------------------------------
loss: 0.334933  [ 2048/60000]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.376030 

Epoch 8
-------------------------------
loss: 0.322334  [ 2048/60000]
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.366672 

Epoch 9
----------------

In [39]:

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer3)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.062024  [ 2048/60000]
Test Error: 
 Accuracy: 89.2%, Avg loss: 0.450177 

Epoch 2
-------------------------------
loss: 0.063572  [ 2048/60000]
Test Error: 
 Accuracy: 89.2%, Avg loss: 0.455957 

Epoch 3
-------------------------------
loss: 0.059535  [ 2048/60000]
Test Error: 
 Accuracy: 89.2%, Avg loss: 0.462431 

Epoch 4
-------------------------------
loss: 0.061663  [ 2048/60000]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.483463 

Epoch 5
-------------------------------
loss: 0.071416  [ 2048/60000]
Test Error: 
 Accuracy: 87.9%, Avg loss: 0.548883 

Epoch 6
-------------------------------
loss: 0.117008  [ 2048/60000]
Test Error: 
 Accuracy: 88.4%, Avg loss: 0.506068 

Epoch 7
-------------------------------
loss: 0.080547  [ 2048/60000]
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.499909 

Epoch 8
-------------------------------
loss: 0.066490  [ 2048/60000]
Test Error: 
 Accuracy: 88.9%, Avg loss: 0.510123 

Epoch 9
----------------

In [40]:
model2 = NeuralNetwork().to(device)
model2.load_state_dict(torch.load('model_weight.pth'))
model2.eval()
torch.save(model2, 'model.pth')